# 주제 : 고양이 얼굴 찾는 모델 - 데이터 준비
---
이미지를 직접 라벨링해서 사용하는 방법을 이해한다.

## 준비 사항
  1. 고양이 데이터 셋 : [Cats Datasets](https://www.kaggle.com/crawford/cat-dataset/code)
  2. 이미지 라벨링 프로그램 [labelImg download](https://github.com/tzutalin/labelImg/releases)
  3. 파일변환 파일 [pascal_to_csv.py](https://gist.github.com/rotemtam/88d9a4efae243fc77ed4a0f9917c8f6c)  


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2


### 문제 1. 데이터 준비

* 데이터셋을 다운 받아서 라벨링을 진행합니다.

In [ ]:
# 라벨링된 데이터를 업로드한후에 실행

In [ ]:
# 압축 풀기
!unzip -qq '/content/cats_bb_sample.zip' -d './datasets'

### 문제 2. 데이터 파일 변환

In [ ]:
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET

In [ ]:
# 이미지가 저장된 폴더

IMAGE_PATH = '/content/datasets/cats_bb_sample'

In [ ]:
# 함수 가져오기 

def xml_to_csv(path):
    print(path)
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            bbx = member.find('bndbox')
            xmin = int(bbx.find('xmin').text)
            ymin = int(bbx.find('ymin').text)
            xmax = int(bbx.find('xmax').text)
            ymax = int(bbx.find('ymax').text)
            label = member.find('name').text

            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     label,
                     xmin,
                     ymin,
                     xmax,
                     ymax
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height',
                   'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df

In [ ]:
# 함수 실행 

xml_df = xml_to_csv(IMAGE_PATH)

csv_path = os.path.join(IMAGE_PATH, 'labels_cats.csv')
xml_df.to_csv(csv_path, index=None)

print('Successfully converted xml to csv. path:', csv_path)

### 문제 3. 변환 데이터 확인

In [ ]:
pd_frame = pd.read_csv(csv_path, sep=',')
pd_frame.head()

###  문제 4. 데이터 시각화 

In [ ]:
# dataset = pd_frame.to_numpy()
# dataset.shape

In [ ]:
images = pd_frame.iloc[:, 0]
images

In [ ]:
# X, Y 좌표들만 끊어서 사용

points = pd_frame.iloc[:, 4:]
points.shape

In [ ]:
points = points.values
points

In [ ]:
# (x, y) 형태로 변환

points = points.reshape(-1, 2, 2)
points


In [ ]:
import cv2
import matplotlib.pyplot as plt


# CV로 이미지를 읽으면, BGR 채녈 
img = cv2.imread(os.path.join(IMAGE_PATH, images[0]))
img = cv2.rectangle(img, tuple(points[0][0]), tuple(points[0][1]), color=(255, 0, 0), thickness=2)

plt.figure(figsize=(16, 6))
plt.subplot(1, 2, 1)
plt.imshow(img)
plt.subplot(1, 2, 2)
plt.imshow(img[:, :, ::-1])    # bgr -> rgb
plt.show()

In [ ]:
# 이미지 여러개 표시 

plt.figure(figsize=(16, 12))
for i in range(4):
    img = cv2.imread(os.path.join(IMAGE_PATH, images[i]))
    img = cv2.rectangle(img, tuple(points[i][0]), tuple(points[i][1]), color=(255, 0, 0), thickness=2)
    plt.subplot(2, 2, i+1)
    plt.imshow(img)
    
plt.show()

### 문제 5. 이미지 사이즈 변경 

In [ ]:
IMG_SIZE = 224

In [ ]:
img = cv2.imread(os.path.join(IMAGE_PATH, images[0]))

plt.figure()
plt.imshow(img)
plt.show()

In [ ]:
# 변경 스케일 지정

old_size = img.shape[:2] #  원본 사이즈 (H, W)
print(old_size)

ratio = float(IMG_SIZE) / max(old_size) # 변경 스케일 계산 
print(ratio)

In [ ]:
# 비율에 맞추어서 사이즈 계산

new_size = tuple([int(x*ratio) for x in old_size])

new_size

In [ ]:
# 사이즈 변경

img = cv2.resize(img, (new_size[1], new_size[0]))  # (W, H)로 지정
print(img.shape)

In [ ]:
plt.figure()
plt.imshow(img)
plt.show()

In [ ]:
# 원본과 변환 후의 크기 차이 계산 

delta_h = IMG_SIZE - new_size[0]
delta_w = IMG_SIZE - new_size[1]


delta_h, delta_w

In [ ]:
top, bottom = delta_h // 2, delta_h - (delta_h // 2)

top, bottom

In [ ]:
left, right = delta_w // 2, delta_w - (delta_w // 2)

left, right

*  [cv2.copyMakeBorder 함수](https://www.geeksforgeeks.org/python-opencv-cv2-copymakeborder-method/) 참조 

In [ ]:
new_img = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=[0, 0, 0])
    
new_img.shape

In [ ]:
plt.figure(figsize=(7, 7))
plt.imshow(new_img)
plt.show()

In [ ]:
# 첫번째 이미지 바운딩 박스 
bbs = points[0]
bbs

In [ ]:
# 바운딩박스 수정

bbs = ((bbs * ratio) + np.array([left, top])).astype(int)

bbs, bbs.dtype

In [ ]:
bbs.shape

In [ ]:
# 표시하기 

plt.figure(figsize=(7, 7))
new_img = cv2.rectangle(new_img, (bbs[0][0], bbs[0][1]), (bbs[1][0], bbs[1][1]), color=(255, 0, 0), thickness=2)
plt.imshow(new_img)
plt.show()

In [ ]:
### 문제 6. 이미지 변경 함수로 만들기

In [ ]:
# 이미지 사이즈를 통일

def resize_img(img):
    old_size = img.shape[:2] #  원본 사이즈 (H, W)
    ratio = float(IMG_SIZE) / max(old_size) # 변경 스케일 계산 
    
    new_size = tuple([int(x*ratio) for x in old_size])
    
    # (width, height) 로 변경 
    img = cv2.resize(img, (new_size[1], new_size[0]))  # (W, H)로 지정
    
    # 두 이미지 사이의 차이
    delta_h = IMG_SIZE - new_size[0]
    delta_w = IMG_SIZE - new_size[1]
    
    # 바운딩박스 변화량
    top, bottom = delta_h // 2, delta_h - (delta_h // 2)
    left, right = delta_w // 2, delta_w - (delta_w // 2)
    
    # 여백 부분 검은색 
    new_img = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT,value=[0, 0, 0])
    return new_img, ratio, top, left



In [ ]:
dataset_images = []
dataset_bbs = []

In [ ]:
# 전체 데이터 처리 

for filename, bbs in zip(images, points):
    
    img = cv2.imread(os.path.join(IMAGE_PATH, filename))
    
    img, ratio, top, left = resize_img(img)
    bbs = ((bbs * ratio) + np.array([left, top])).astype(int)
    
    dataset_images.append(img)
    dataset_bbs.append(bbs.flatten())

In [ ]:
dataset_images = np.array(dataset_images)
dataset_bbs = np.array(dataset_bbs)

dataset_images.shape, dataset_bbs.shape

### 문제 6. 데이터 저장 

In [ ]:
np.savez('cat_bbs.npz', image=dataset_images, bbs=dataset_bbs)

### 문제 7. 데이터 파일 읽어오기

In [ ]:
dataset = np.load('cat_bbs.npz')

dataset['image'].shape, dataset['bbs'].shape